In [1]:
import numpy as np
import csv
import pandas as pd
import time 
import LESbplume as lp

In [2]:
#(r"D:\Ammonia_NBL_3D\D:\Documents\Ammonia_NBL_3D\Ammonia_NBL_3D,skiprows = sk,nrows = n, dtype =np.float32,header = None)
#(r"D:\Helium_FLUX_1\He_h01_01_T.csv",skiprows = sk,nrows = n, dtype =np.float32)

In [3]:
Start = 0 # numer of rows in dataset
End = 4 # time start

sk = Start*45*45*700
nr = End - Start
n = 45*45*700*nr
name = 'Ammonia'
dx=dy=40
dz =10

l2 = pd.read_csv(r"C:\Users\thanh\Documents\MATLAB\lambda2_Sam.csv",skiprows = sk,nrows = n, dtype =np.float32,header = None)
lamb2sam = l2.to_numpy()
del l2
lamb2sam = np.reshape(lamb2sam, (45,45,700,nr), order="F")

dfu = pd.read_csv(r"D:\Documents\\Helium_2\Helium_2_U.csv",skiprows = sk,nrows = n, dtype =np.float32,header = None)
datau = dfu.to_numpy()
del dfu
datau = np.reshape(datau, (45,45,700,nr), order="F")

dfv = pd.read_csv(r"D:\Documents\\Helium_2\Helium_2_V.csv",skiprows = sk,nrows = n, dtype =np.float32,header = None)
datav = dfv.to_numpy()
del dfv
datav = np.reshape(datav, (45,45,700,nr), order="F")

dfw = pd.read_csv(r"D:\Documents\\Helium_2\Helium_2_W.csv",skiprows = sk,nrows = n, dtype =np.float32,header = None)
dataw = dfw.to_numpy()
del dfw
dataw = np.reshape(dataw, (45,45,700,nr), order="F")

In [10]:
# #Calculate Lambda2 
# #Estimated time: 2h-3h
# #Input (u,v,w,dx,dy,dz)
# tic = time.perf_counter()
# lamb2 = fl.lambda2t(datau,datav,dataw,dx=40,dy=40,dz=10)
# toc1 = time.perf_counter()
# print(f"Time: {((toc1 - tic)/60):0.4f} minutes")

In [4]:
from numpy import linalg as LA
import numpy as np
def shape(dt):
    return dt.shape[0],dt.shape[1],dt.shape[2],dt.shape[3]
def transpose(x): 
    a11 = x[0,0]
    a12 = x[0,1]
    a13 = x[0,2]
    a21 = x[1,0]
    a22 = x[1,1]
    a23 = x[1,2]
    a31 = x[2,0]
    a32 = x[2,1]
    a33 = x[2,2]
    x = np.array([[a11,a21,a31],[a12,a22,a32],[a13,a23,a33]])
    return x
def muiltiply(x,y):
    a11 = x[0,0]*y[0,0]+x[0,1]*y[1,0]+x[0,2]*y[2,0]
    a12 = x[0,0]*y[0,1]+x[0,1]*y[1,1]+x[0,2]*y[2,1]
    a13 = x[0,0]*y[0,2]+x[0,1]*y[1,2]+x[0,2]*y[2,2]
    a21 = x[1,0]*y[0,0]+x[1,1]*y[1,0]+x[1,2]*y[2,0]
    a22 = x[1,0]*y[0,1]+x[1,1]*y[1,1]+x[1,2]*y[2,1]
    a23 = x[1,0]*y[0,2]+x[1,1]*y[1,2]+x[1,2]*y[2,2]
    a31 = x[2,0]*y[0,0]+x[2,1]*y[1,0]+x[2,2]*y[2,0]
    a32 = x[2,0]*y[0,1]+x[2,1]*y[1,1]+x[2,2]*y[2,1]
    a33 = x[2,0]*y[0,2]+x[2,1]*y[1,2]+x[2,2]*y[2,2]
    x = np.array([[a11,a21,a31],[a12,a22,a32],[a13,a23,a33]])
    return x
#trace of 3x3 matrix(AT A)
def trace(x):
    tra = (x[0,0]*x[1,1]*x[2,2])
    return tra
def lambmatrix(S,Ome,nx,ny,nz):
    from numpy import linalg as LA
    lamb2 = np.zeros((nx,ny,nz),dtype=np.float32)
    for i in range(0,nx):
                for j in range(0,ny):
                    for k in range(0,nz):
                        matrix = S[:,:,i,j,k].dot(S[:,:,i,j,k])+Ome[:,:,i,j,k].dot(Ome[:,:,i,j,k])
                        w = LA.eigvals(matrix)
                        w= np.sort(w)
                        if w[1]<0:
                            lamb2[i,j,k] = w[1]
    return lamb2
def lambda2(datau,datav,dataw,t,dx=40,dy=40,dz=10):
    from numpy import linalg as LA
    nx,ny,nz,nt = shape(datau)         
    print(t)
        
    gu = np.gradient(datau[:,:,:,t],dx,dy,dz)
    gv = np.gradient(datav[:,:,:,t],dx,dy,dz)
    gw = np.gradient(dataw[:,:,:,t],dx,dy,dz)             
    J = np.array([gu,gv,gw])
    JT = transpose(J)
    S = (J+JT)*0.5
    Ome = (J-JT)*0.5
    lamb2 = lambmatrix(S,Ome,nx,ny,nz)
#     print('{}\r'.format("100% Completed"), end="")
    return lamb2

In [ ]:
# t =1
# lamb2test = lp.lambda2t(datau,datav,dataw,dx,dy,dz)

In [4]:
# def lambda2(datau,datav,dataw,t,dx=40,dy=40,dz=10):

t = 1
tic = time.perf_counter()
from numpy import linalg as LA
nx,ny,nz,nt = lp.shape(datau)         
print(t)    
dUdy,dUdx,dUdz = np.gradient(datau[:,:,:,t],dx,dy,dz)
dVdy,dVdx,dVdz = np.gradient(datav[:,:,:,t],dx,dy,dz)
dWdy,dWdx,dWdz = np.gradient(dataw[:,:,:,t],dx,dy,dz)
lamb2 = np.zeros((nx,ny,nz),dtype = np.float32)
for i in range(0,nx):
        for j in range(0,ny):
            for k in range(0,nz):
                dudx=dUdx[i,j,k]
                dudy=dUdy[i,j,k]
                dudz=dUdz[i,j,k]
                dvdx=dVdx[i,j,k]
                dvdy=dVdy[i,j,k]
                dvdz=dVdz[i,j,k]
                dwdx=dWdx[i,j,k]
                dwdy=dWdy[i,j,k]
                dwdz=dWdz[i,j,k]
                Om = np.array([[0,dudy-dvdx,dudz-dwdx],
                      [dvdx-dudy,0,dvdz-dwdy],
                      [dwdx-dudz,dwdy-dvdz,0]])
                Om = Om*0.5
                S  = np.array([[2*dudx,dudy+dvdx,dudz+dwdx],
                      [dvdx+dudy,2*dvdy,dvdz+dwdy],
                      [dwdx+dudz,dwdy+dvdz,2*dwdz]])
                S = S*0.5
                matrix = S.dot(S) + Om.dot(Om)
                w = LA.eigvals(matrix)
                w= np.sort(w)
                if w[1]<0:
                    lamb2[i,j,k] = w[1]
#     return lamb2
toc1 = time.perf_counter()
print(f"Time: {((toc1 - tic)/60):0.4f} minutes")

1
Time: 1.3949 minutes


In [5]:
# lamb2test[:,22,:,t]

NameError: name 'lamb2test' is not defined

In [6]:
lamb2sam[:,22,:,t]

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -1.4792011e-11,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00, -3.5463373e-12],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.3481896e-11, -3.1956196e-10, -8.8617497e-10],
       ...,
       [ 0.0000000e+00,  0.0000000e+00, -1.4779304e-10, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -2.3594604e-11, -1.0581950e-11,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -9.5671068e-11, -2.3561286e-10, -4.7336257e-10]], dtype=float32)

In [7]:
lamb2[:,22,:]

array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -1.4792015e-11,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00, -3.5463325e-12],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.3481889e-11, -3.1956193e-10, -8.8617491e-10],
       ...,
       [ 0.0000000e+00,  0.0000000e+00, -1.4779303e-10, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -2.3594604e-11, -1.0581947e-11,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -9.5671061e-11, -2.3561289e-10, -4.7336263e-10]], dtype=float32)

In [35]:
t = 2
tic = time.perf_counter()
from numpy import linalg as LA
nx,ny,nz,nt = lp.shape(datau)         
print(t)
x = [40,40,10]
U = datau[:,:,:,t]
dUdy,dUdx,dUdz = np.gradient(U,dx,dy,dz)
dVdy,dVdx,dVdz = np.gradient(datav[:,:,:,t],dx,dy,dz)
dWdy,dWdx,dWdz = np.gradient(dataw[:,:,:,t],dx,dy,dz)
print(dWdy[22,22,99])
print(U[22,22,99])
# lamb2 = np.zeros((nx,ny,nz),dtype = np.float32)
# for i in range(0,nx):
#         for j in range(0,ny):
#             for k in range(0,nz):
#                 dudx=dUdx[i,j,k]
#                 dudy=dUdy[i,j,k]
#                 dudz=dUdz[i,j,k]
#                 dvdx=dVdx[i,j,k]
#                 dvdy=dVdy[i,j,k]
#                 dvdz=dVdz[i,j,k]
#                 dwdx=dWdx[i,j,k]
#                 dwdy=dWdy[i,j,k]
#                 dwdz=dWdz[i,j,k]
#                 Om = np.array([[0,dudy-dvdx,dudz-dwdx],
#                       [dvdx-dudy,0,dvdz-dwdy],
#                       [dwdx-dudz,dwdy-dvdz,0]])
#                 Om = Om*0.5
#                 S  = np.array([[2*dudx,dudy+dvdx,dudz+dwdx],
#                       [dvdx+dudy,2*dvdy,dvdz+dwdy],
#                       [dwdx+dudz,dwdy+dvdz,2*dwdz]])
#                 S = S*0.5
#                 matrix = S.dot(S) + Om.dot(Om)
#                 w = LA.eigvals(matrix)
#                 w= np.sort(w)
#                 if w[1]<0:
#                     lamb2[i,j,k] = w[1]
# #     return lamb2
# toc1 = time.perf_counter()
# print(f"Time: {((toc1 - tic)/60):0.4f} minutes")

2
-1.3419995e-06
-7.40591e-06


In [63]:
x = [1,2,3,4,5,6,7,8,9,10]
fx = np.gradient(x,2)
fx

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [30]:
dudy,dudx,dudz= np.gradient(datau[:,:,:,t],dx,dy,dz)
dvdy,dvdx,dvdz = np.gradient(datav[:,:,:,t],dx,dy,dz)
dwdy,dwdx,dwdz = np.gradient(dataw[:,:,:,t],dx,dy,dz)

J = np.array([[dudx,dudy,dudz],[dvdx,dvdy,dvdz],[dwdx,dwdy,dwdz]])

In [31]:
J

array([[2, 1, 3],
       [5, 4, 6],
       [8, 7, 9]])

In [27]:
J[2,2,22,22,99]

-1.3419995e-06